In [1]:
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np

In [2]:
simplify_dir = 'GTFS/stop_snapping/GIS Simplify/'
gtfs_dir = 'GTFS/TTC_2016-10-03/'

In [3]:
stop_times_full = pd.read_csv(gtfs_dir + 'stop_times_reduced.csv')
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [4]:
stop_times_full

,trip_id,stop_sequence_new,stop_sequence,hr,min,INT_ID
0,32828689,1,1,5,15,13466640.0
1,32828689,2,2,5,16,13466531.0
2,32828689,3,3,5,16,13466677.0
3,32828689,4,4,5,17,13466754.0
4,32828689,5,5,5,18,13466699.0
...,...,...,...,...,...,...
3959705,32970576,7,8,25,56,13461602.0
3959706,32970576,8,9,25,57,13461912.0
3959707,32970576,9,10,25,57,13462284.0
3959708,32970576,10,11,25,58,13462427.0


In [5]:
stop_times_full['hr'] = np.where(stop_times_full['hr'] > 23, stop_times_full['hr'] - 24, stop_times_full['hr'])
stop_times_full_o = stop_times_full.copy()
stop_times_full_d = stop_times_full.copy()
stop_times_full_d['stop_sequence_new'] = stop_times_full_d['stop_sequence_new'] - 1
stop_times_od = stop_times_full_o.merge(stop_times_full_d, left_on = ['trip_id', 'stop_sequence_new'],
                           right_on = ['trip_id', 'stop_sequence_new'], suffixes = ['_o', '_d'])
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']])

In [6]:
stop_times_od.groupby(['hr_o', 'INT_ID_o', 'INT_ID_d']).count().reset_index().sort_values(
    by = 'trip_id', ascending = False)[['INT_ID_o', 'INT_ID_d', 'trip_id']].drop_duplicates(
    subset = ['INT_ID_o', 'INT_ID_d'], keep = 'first').head(10)

,INT_ID_o,INT_ID_d,trip_id
107013,304.0,305.0,84
46283,305.0,304.0,65
49123,13452476.0,228.0,61
38448,228.0,13452476.0,60
53043,13468120.0,13468053.0,51
53063,13468224.0,13468120.0,51
52974,13467722.0,13467654.0,50
52953,13467654.0,13467505.0,50
53034,13468053.0,13467971.0,50
38246,116.0,13445250.0,49


In [7]:
freq_df = stop_times_od[['hr_o', 'INT_ID_o','INT_ID_d', 'trip_id']]

freq_df = freq_df.groupby(['hr_o', 'INT_ID_o','INT_ID_d']).count().reset_index().rename(columns = {'trip_id':'freq'})
freq_df_r = freq_df.copy()
freq_df_r = freq_df_r.rename(columns = {'INT_ID_o':'INT_ID_d', 'INT_ID_d':'INT_ID_o', 'freq':'freq_r'})
freq_df = freq_df.merge(freq_df_r, how = 'outer')
freq_df['freq'] = freq_df['freq'].fillna(0)
freq_df['freq_r'] = freq_df['freq_r'].fillna(0)
freq_df['freq_tot'] = freq_df['freq_r'] + freq_df['freq']

freq_df = freq_df[['hr_o', 'INT_ID_o', 'INT_ID_d', 'freq_tot']]
freq_df

,hr_o,INT_ID_o,INT_ID_d,freq_tot
0,0,100.0,101.0,27.0
1,0,100.0,151.0,25.0
2,0,100.0,13467163.0,2.0
3,0,100.0,13467239.0,8.0
4,0,100.0,13467593.0,8.0
...,...,...,...,...
195445,23,13469683.0,30070002.0,2.0
195446,23,13469442.0,30070005.0,2.0
195447,23,13445039.0,30090087.0,1.0
195448,23,13467715.0,30109241.0,9.0


In [8]:
freq_df = freq_df[~freq_df.T.apply(sorted).T.duplicated()]
freq_df.sort_values(by = 'freq_tot', ascending = False)

,hr_o,INT_ID_o,INT_ID_d,freq_tot
46277,8,304.0,305.0,145.0
114791,17,304.0,305.0,145.0
122594,18,304.0,305.0,145.0
107013,16,304.0,305.0,144.0
99269,15,304.0,305.0,138.0
...,...,...,...,...
19087,3,13452977.0,13453094.0,1.0
93178,14,13447223.0,13449708.0,1.0
21236,4,13444194.0,13444493.0,1.0
157397,22,13460422.0,13460183.0,1.0


In [9]:
max_freq = freq_df['freq_tot'].max()
max_freq

145.0

In [10]:
stop_times_od[(stop_times_od['INT_ID_o'] == 304) & (stop_times_od['INT_ID_d'] == 305)][['route_short_name']].drop_duplicates()

,route_short_name
50561,134
199941,129
209275,131
209949,9131
213980,132
219108,133
234778,16
278222,169
402693,3
515067,38


In [11]:
stop_times_od

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,route_short_name,direction_id
0,32828689,1,1,5,15,13466640.0,2,5,16,13466531.0,506,0
1,32828689,2,2,5,16,13466531.0,3,5,16,13466677.0,506,0
2,32828689,3,3,5,16,13466677.0,4,5,17,13466754.0,506,0
3,32828689,4,4,5,17,13466754.0,5,5,18,13466699.0,506,0
4,32828689,5,5,5,18,13466699.0,6,5,19,14043045.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1213714,32970488,6,7,3,4,13461421.0,8,3,4,13461602.0,306,1
1213715,32970488,7,8,3,4,13461602.0,9,3,5,13461912.0,306,1
1213716,32970488,8,9,3,5,13461912.0,10,3,5,13462284.0,306,1
1213717,32970488,9,10,3,5,13462284.0,11,3,6,13462427.0,306,1


In [12]:
trips_weekday = pd.read_csv(gtfs_dir + 'trips_weekday.csv')

In [13]:
stop_times_od = stop_times_od.merge(trips_weekday[['trip_id', 'route_short_name', 'direction_id']])

In [14]:
stop_times_od 

,trip_id,stop_sequence_new,stop_sequence_o,hr_o,min_o,INT_ID_o,stop_sequence_d,hr_d,min_d,INT_ID_d,route_short_name,direction_id
0,32828689,1,1,5,15,13466640.0,2,5,16,13466531.0,506,0
1,32828689,2,2,5,16,13466531.0,3,5,16,13466677.0,506,0
2,32828689,3,3,5,16,13466677.0,4,5,17,13466754.0,506,0
3,32828689,4,4,5,17,13466754.0,5,5,18,13466699.0,506,0
4,32828689,5,5,5,18,13466699.0,6,5,19,14043045.0,506,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1213714,32970488,6,7,3,4,13461421.0,8,3,4,13461602.0,306,1
1213715,32970488,7,8,3,4,13461602.0,9,3,5,13461912.0,306,1
1213716,32970488,8,9,3,5,13461912.0,10,3,5,13462284.0,306,1
1213717,32970488,9,10,3,5,13462284.0,11,3,6,13462427.0,306,1


In [15]:
def graph_builder(hr_list, period):
    
    stop_times_od_func = stop_times_od[stop_times_od['hr_o'].isin(hr_list)].copy()
    
    route_map_links = stop_times_od_func.groupby(['INT_ID_o', 'INT_ID_d']
                     ).count().reset_index()[['INT_ID_o','INT_ID_d', 'trip_id']]
    route_map_links = route_map_links.rename(columns = {'trip_id':'freq'})
    
    
    route_map_links_reverse = route_map_links.rename(columns = {'INT_ID_o':'INT_ID_d', 
                                                                'INT_ID_d':'INT_ID_o', 'freq':'freq_r'})
    
    #merging and removing 0s
    route_map_links = route_map_links.merge(route_map_links_reverse, how = 'outer')
    route_map_links['freq'] = route_map_links['freq'].fillna(0)
    
    route_map_links['freq_r'] = route_map_links['freq_r'].fillna(0)
    
    
    route_map_links['freq_tot'] = route_map_links['freq'] + route_map_links['freq_r']
    
    #remove duplicates
    route_map_links[~route_map_links.T.apply(sorted).T.duplicated()]
    
    route_map_links = route_map_links[['INT_ID_o', 'INT_ID_d', 'freq_tot']]
    
    route_map_links['freq_tot'] = route_map_links['freq_tot']/2
        
    # adding a dummy link so networkx scales properly
    route_map_links = route_map_links.append(pd.DataFrame.from_records([[1,2, max_freq]], columns = ['INT_ID_o', 'INT_ID_d','freq_tot']))
    route_map_links = route_map_links.reset_index().drop(columns = ['index'])
    
    route_map_links['scaled_freq'] = route_map_links['freq_tot']/max_freq
    
    route_map_links['INT_ID_o'] = route_map_links['INT_ID_o'].astype(int)
    route_map_links['INT_ID_d'] = route_map_links['INT_ID_d'].astype(int)
    
    route_map_links['link'] =  route_map_links['INT_ID_o'].astype(str) + '-' + route_map_links['INT_ID_d'].astype(str)
    
    G = nx.Graph()
    
    nodes = list((route_map_links['INT_ID_o'].append(route_map_links['INT_ID_d'])).drop_duplicates())
    
    G.add_nodes_from(nodes)
    
    link_list = []
    for index, row in route_map_links.iterrows():
        attributes = {'link_id':row['link'], 
                      'freq':row['freq_tot'], 'scaled_freq': row['scaled_freq']}
        link = (row['INT_ID_o'], row['INT_ID_d'], attributes)
        link_list.append(link)
        
    G.add_edges_from(link_list)
    
    nx.write_gexf(G, 'networks/' + period + '-RM-16.gexf')
    
    route_map_links.to_csv('networks/' + period + '-RM-16.csv', index = False)

In [16]:
period_list = [['EM', [3,4]], ['AM', [7,8]], ['MD', [11,12]], ['PM', [16,17]], ['EV', [20,21]]]

In [17]:
for i in period_list:
    hr_list = i[1]
    graph_builder(hr_list, i[0])